In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ----------------------------
# Basic 3x3 Convolution
# ----------------------------
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

# ----------------------------
# Basic Residual Block
# ----------------------------
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, downsample=None, is_last=False):
        super().__init__()
        self.is_last = is_last
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        preact = out
        out = F.relu(out)

        if self.is_last:
            return out, preact
        else:
            return out

# ----------------------------
# ResNet CIFAR Modular
# ----------------------------
class ResNet(nn.Module):
    def __init__(self, depth, num_filters=[16,16,32,64], block_name='basicblock', num_classes=100):
        super().__init__()
        assert block_name.lower() == 'basicblock', "Currently only BasicBlock supported"
        assert (depth - 2) % 6 == 0, "Depth must be 6n+2 for BasicBlock"
        n = (depth - 2) // 6

        self.in_planes = num_filters[0]
        self.conv1 = conv3x3(3, num_filters[0])
        self.bn1 = nn.BatchNorm2d(num_filters[0])
        self.relu = nn.ReLU(inplace=True)

        # Residual layers
        self.layer1 = self._make_layer(BasicBlock, num_filters[1], n)
        self.layer2 = self._make_layer(BasicBlock, num_filters[2], n, stride=2)
        self.layer3 = self._make_layer(BasicBlock, num_filters[3], n, stride=2)

        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(num_filters[3]*BasicBlock.expansion, num_classes)

        # weight init
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes*block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes*block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes*block.expansion)
            )

        layers = [block(self.in_planes, planes, stride, downsample, is_last=(blocks==1))]
        self.in_planes = planes*block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes, is_last=(i==blocks-1)))
        return nn.Sequential(*layers)

    def forward(self, x, is_feat=False, preact=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        f0 = x

        x, f1_pre = self.layer1(x)
        f1 = x
        x, f2_pre = self.layer2(x)
        f2 = x
        x, f3_pre = self.layer3(x)
        f3 = x

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        f4 = x
        x = self.fc(x)

        if is_feat:
            return ([f0,f1,f2,f3,f4], x) if not preact else ([f0,f1_pre,f2_pre,f3_pre,f4], x)
        else:
            return x

# ----------------------------
# Architecture builders
# ----------------------------
def resnet20(num_classes=100): return ResNet(20, num_classes=num_classes)
def resnet32(num_classes=100):
    """ResNet32 matching the standard CIFAR ResNet paper structure."""
    # Use num_filters=[32, 32, 64, 128] to match the checkpoint you want to load
    return ResNet(depth=32, num_filters=[32, 64, 128, 256], block_name='basicblock', num_classes=num_classes)

def resnet56(num_classes=100): return ResNet(56, num_classes=num_classes)
def resnet110(num_classes=100): return ResNet(110, num_classes=num_classes)
def resnet32_basic(num_classes=100): return ResNet(32, num_classes=num_classes)

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------------------------
# Device
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------------
# CIFAR-100 test dataset
# ---------------------------
mean = (0.5071, 0.4867, 0.4408)
std  = (0.2675, 0.2565, 0.2761)

test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_ds = datasets.CIFAR100("./data", train=False, transform=test_tf, download=True)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# ---------------------------
# Model constructors
# ---------------------------
model_dict = {
    # "ResNet20": resnet20,
    # "ResNet32": resnet32(), # this is the resnet-32x4
    "ResNet56": resnet56(),
    "ResNet110": resnet110()
}

# Path mapping for pretrained weights (adjust paths if needed)
weights_dict = {
    # "ResNet20": "/path/to/resnet20.pth",
    # "ResNet32": "/kaggle/input/resnet32/pytorch/default/1/ckpt_epoch_240.pth",
    "ResNet56":  "/kaggle/input/resnet-56/ckpt_epoch_240.pth",
    "ResNet110": "/kaggle/input/resnet-110/ckpt_epoch_240.pth"
}

# ---------------------------
# Evaluation loop
# ---------------------------
for name, constructor in model_dict.items():
    print(f"Evaluating {name}...")
    model = constructor.to(device)
    
    # Load pretrained weights if available
    weight_path = weights_dict.get(name)
    if weight_path:
        checkpoint = torch.load(weight_path, map_location=device,weights_only=False)
        if 'model' in checkpoint:
            state_dict = checkpoint['model']
        else:
            state_dict = checkpoint
        model.load_state_dict(state_dict)
    
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_acc = 100 * correct / total
    print(f"🎯 {name} Test Accuracy on CIFAR-100: {test_acc:.2f}%\n")


Evaluating ResNet56...
🎯 ResNet56 Test Accuracy on CIFAR-100: 72.41%

Evaluating ResNet110...
🎯 ResNet110 Test Accuracy on CIFAR-100: 74.31%



# Training the student models BASIC Resnet 20 / 32

import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy, RandomErasing
from tqdm.auto import tqdm
#from torchvision.models import resnet18, resnet34



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_dp = torch.cuda.device_count() > 1
torch.manual_seed(42)
np.random.seed(42)

mean = (0.5071, 0.4867, 0.4408)  # CIFAR-100 mean
std  = (0.2675, 0.2565, 0.2761)  # CIFAR-100 std

batch_size = 64

# Precompute DataLoaders for each resolution
stages = [(r) for r in [(32, 240)]]
dataloader_dict = {}

'''
Copied from the paper as it is.

they are not using any validation sets. Training it on the entire train set!
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=stdv),
    ])
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=stdv),
    ])
    
    '''

for resolution, _ in stages:
    train_tf = transforms.Compose([
        transforms.RandomCrop(resolution,padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    
    train_set = datasets.CIFAR100('./data', train=True, download=False, transform=train_tf)
    
    train_loader = DataLoader(train_set, batch_size=batch_size,
                              shuffle=True, num_workers=0, pin_memory=True)
    dataloader_dict[resolution] = {
        'train': train_loader
    }

# Test loader (fixed resolution)
test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_ds = datasets.CIFAR100('./data', train=False, transform=test_tf)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


# learning_rate is divided by 10 

# ---------------------------
# Test loop
# ---------------------------

def test(Test_model):
    
    Test_model.eval()
    correct_val = total_val = 0
    
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = Test_model(imgs)
            preds = logits.argmax(1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    test_acc = 100 * correct_val / total_val

    # print(f"Test Acc = {val_acc:.2f}%")
    return test_acc

# ---------------------------
# Training loop
# ---------------------------

def train(model, model_type):
    
    # Loss + optimizer
    lr = 0.05 # as per the paper
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    # Step LR schedule: decay at 150, 180, 210 epochs
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 180, 210], gamma=0.1)

    best_val_acc = 0.0
    num_epochs = 240
    
    for res, epochs in stages:
        
        print(f"\n=== Training at resolution {res}px ===")
        tr_loader = dataloader_dict[res]['train']
        
        for e in range(1, epochs+1):
            
            model.train()
            total_loss = 0
            correct = total = 0
        
            for imgs, labels in tqdm(tr_loader, desc=f"Epoch {e}/{num_epochs}"):
                
                imgs, labels = imgs.to(device), labels.to(device)
                optimizer.zero_grad()
                logits = model(imgs)
                loss = criterion(logits, labels)
                loss.backward()
                optimizer.step()
        
                total_loss += loss.item()
                preds = logits.argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
            train_acc = 100 * correct / total
            scheduler.step()

            test_acc = test(model)
            
            print(f"Epoch {e}: Train Acc = {train_acc:.2f}% Test Accuracy = {test_acc:.2f}%") 
            
            # Save best model
            if test_acc > best_val_acc:
                best_val_acc = test_acc
                torch.save(model.state_dict(), f"resnet{model_type}_student.pth")
                print(f"→ Saved best model at epoch {e} with Test Acc = {train_acc:.2f}%")
    print("✅ Training completed!")

# KD Training Vanilla


In [3]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy, RandomErasing
from tqdm.auto import tqdm
#from torchvision.models import resnet18, resnet34
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


mean = (0.5071, 0.4867, 0.4408)
std  = (0.2675, 0.2565, 0.2761)
batch_size = 128
num_workers = 0
pin_mem = True

@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    correct = total = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return 100.0 * correct / total


torch.manual_seed(27)
np.random.seed(27)

torch.cuda.manual_seed_all(27)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True



In [4]:
stages = [(r) for r in [(32, 240)]]
dataloader_dict = {}

for resolution, _ in stages:
    train_tf = transforms.Compose([
        transforms.RandomCrop(resolution, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    train_set = datasets.CIFAR100('./data', train=True, download=True, transform=train_tf)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=pin_mem)

    dataloader_dict[resolution] = {'train': train_loader}


test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_set = datasets.CIFAR100('./data', train=False, download=True, transform=test_tf)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=pin_mem)


resolution, _ = stages[0]          # (32, 240)
train_loader = dataloader_dict[resolution]['train']


In [5]:
# learning_rate is divided by 10 
# ---------------------------
# Training loop
# ---------------------------

def kd_loss(student_logits, teacher_logits, labels, T=4.0, alpha=0.9): 
    """
    alpha = 0.1 as per the github repo
    Compute KD loss = α * KD + (1-α) * CE
    T = temperature
    α = weight for soft distillation loss
    """
    # Hard-label loss
    ce = F.cross_entropy(student_logits, labels)
    kd = F.kl_div(
        F.log_softmax(student_logits / T, dim=1),
        F.softmax(teacher_logits / T, dim=1),
        reduction="batchmean") * (T * T)
    
    return (1 - alpha) * ce + alpha * kd

In [6]:
def train_via_KD(t_model, s_model, train_loader, test_loader, device,
                 epochs=240, base_lr=0.05, wd=5e-4, milestones=(150,180,210),
                 T=4.0, alpha=0.9, save_path="student_kd.pth"):

    # Freeze teacher
    t_model.to(device).eval()
    for p in t_model.parameters():
        p.requires_grad = False

    s_model = s_model.to(device)

    optimizer = optim.SGD(s_model.parameters(), lr=base_lr, momentum=0.9, weight_decay=wd)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=list(milestones), gamma=0.1)

    best_test = -1.0
    for e in range(1, epochs + 1):
        s_model.train()
        running_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {e}/{epochs}"):
            imgs, labels = imgs.to(device), labels.to(device)

            with torch.no_grad():
                t_logits = t_model(imgs)

            s_logits = s_model(imgs)
            loss = kd_loss(s_logits, t_logits, labels, T=T, alpha=alpha)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            preds = s_logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        scheduler.step()
        train_loss = running_loss / total
        train_acc  = 100.0 * correct / total
        test_acc   = evaluate(s_model, test_loader, device)

        print(f"Epoch {e:3d}/{epochs} | loss {train_loss:.3f} | train {train_acc:5.2f}% | test {test_acc:5.2f}%")

        if test_acc > best_test:
            best_test = test_acc
            to_save = s_model.module.state_dict() if isinstance(s_model, torch.nn.DataParallel) else s_model.state_dict()
            torch.save(to_save, save_path)
            print(f"  ↳ Saved best @ epoch {e} (test {best_test:.2f}%) → {save_path}")

    print(f"✅ KD training finished. Best Test Acc: {best_test:.2f}%")


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
teacher = resnet56(num_classes = 100)
t_ckpt  = torch.load("/kaggle/input/resnet-56/ckpt_epoch_240.pth", map_location=device, weights_only=False)
teacher.load_state_dict(t_ckpt['model'] if 'model' in t_ckpt else t_ckpt)

student = resnet20(num_classes=100)

train_via_KD(teacher, student, train_loader, test_loader, device, save_path= "56_t-20_s.pth")


teacher = resnet110(num_classes = 100)
t_ckpt  = torch.load("/kaggle/input/resnet-110/ckpt_epoch_240.pth", map_location=device, weights_only=False)
teacher.load_state_dict(t_ckpt['model'] if 'model' in t_ckpt else t_ckpt)

student = resnet20(num_classes=100)

train_via_KD(teacher, student, train_loader, test_loader, device, save_path= "110_t-20_s.pth")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
teacher = resnet110(num_classes = 100)
t_ckpt  = torch.load("/kaggle/input/resnet-110/ckpt_epoch_240.pth", map_location=device, weights_only=False)
teacher.load_state_dict(t_ckpt['model'] if 'model' in t_ckpt else t_ckpt)

student = resnet32_basic(num_classes=100)

train_via_KD(teacher, student, train_loader, test_loader, device, save_path= "110_t-32_s.pth")

# DIFF KD training - latent diffusion for training


In [7]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm


def sinusoidal_embedding(t: torch.Tensor, dim: int) -> torch.Tensor:
    """[B] -> [B, dim]"""
    device = t.device
    half = dim // 2
    freqs = torch.exp(-math.log(10000) * torch.arange(half, device=device).float() / max(half,1))
    args = t.float().unsqueeze(1) * freqs.unsqueeze(0)
    emb = torch.cat([torch.sin(args), torch.cos(args)], dim=1)
    if dim % 2: emb = F.pad(emb, (0,1))
    return emb


@torch.no_grad()
def ddim_update(x_t, eps_pred, abar_t, abar_prev):
    # x0 = (x_t - sqrt(1-abar_t)*eps) / sqrt(abar_t)
    x0 = (x_t - torch.sqrt(1.0 - abar_t) * eps_pred) / torch.sqrt(abar_t)
    x_prev = torch.sqrt(abar_prev) * x0 + torch.sqrt(1.0 - abar_prev) * eps_pred
    return x_prev

class DiffSchedule:
    def __init__(self, T=1000, device='cuda'):
        # linear beta works well for features/logits here
        betas = torch.linspace(1e-4, 2e-2, T, device=device)
        alphas = 1.0 - betas
        self.abar = torch.cumprod(alphas, dim=0)              # [T]
        self.T = T
        self.device = device

    def abar_at(self, t):  # t: Long[B] or scalar int
        return self.abar[t]

# ---------- Tiny denoisers ----------
class Bottleneck2D(nn.Module):
    def __init__(self, c, hidden=None):
        super().__init__()
        h = hidden or max(32, c // 4)
        self.net = nn.Sequential(
            nn.Conv2d(c, h, 1, bias=False), nn.BatchNorm2d(h), nn.ReLU(inplace=True),
            nn.Conv2d(h, h, 3, padding=1, bias=False), nn.BatchNorm2d(h), nn.ReLU(inplace=True),
            nn.Conv2d(h, c, 1, bias=False), nn.BatchNorm2d(c)
        )
        self.act = nn.ReLU(inplace=True)
    def forward(self, x):  # residual
        return self.act(self.net(x) + x)

class DiffusionDenoiser2D(nn.Module):
    """Φ_theta for feature maps [B,C,H,W]. Two bottlenecks + time conditioning."""
    def __init__(self, c, t_dim=128):
        super().__init__()
        self.t_mlp = nn.Sequential(nn.Linear(t_dim, 2*c), nn.SiLU(), nn.Linear(2*c, 2*c))
        self.bn = nn.BatchNorm2d(c)
        self.block1 = Bottleneck2D(c)
        self.block2 = Bottleneck2D(c)
        self.proj = nn.Conv2d(c, c, 1)
        self.t_dim = t_dim
        nn.init.zeros_(self.proj.weight)

    def forward(self, zt, t):
        # time -> FiLM
        temb = self.t_mlp(sinusoidal_embedding(t, self.t_dim))  # [B, 2C]
        B, C, H, W = zt.shape
        gamma, beta = temb[:, :C].view(B,C,1,1), temb[:, C:].view(B,C,1,1)
        h = self.bn(zt) * (1 + gamma) + beta
        h = self.block1(h)
        h = self.block2(h)
        eps = self.proj(h)
        return eps

class DiffusionDenoiser1D(nn.Module):
    """MLP denoiser for logits [B,num_classes]."""
    def __init__(self, d, t_dim=64, hidden=512):
        super().__init__()
        self.t_mlp = nn.Sequential(nn.Linear(t_dim, hidden), nn.SiLU(), nn.Linear(hidden, hidden))
        self.fc1 = nn.Linear(d, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.out = nn.Linear(hidden, d)
        nn.init.zeros_(self.out.weight)

        self.t_dim = t_dim

    def forward(self, zt, t):
        temb = self.t_mlp(sinusoidal_embedding(t, self.t_dim))  # [B,H]
        h = self.fc1(zt) + temb
        h = F.silu(h)
        h = F.silu(self.fc2(h))
        eps = self.out(h)
        return eps

# ---------- Noise adapters (γ) ----------
class NoiseAdapter2D(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv = nn.Conv2d(c, max(8, c//8), 1)
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(max(8, c//8), 1)
        )
    def forward(self, z):    # [B,C,H,W] -> [B,1,1,1] in (0,1)
        g = torch.sigmoid(self.head(F.silu(self.conv(z))))
        return g.view(-1,1,1,1)

class NoiseAdapter1D(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(d, max(8, d//8)), nn.SiLU(), nn.Linear(max(8, d//8), 1))
    def forward(self, z):    # [B,D] -> [B,1] in (0,1)
        return torch.sigmoid(self.net(z))

# ---------- DiffKD trainer ----------
class DiffKDTrainer(nn.Module):
    def __init__(self, teacher, student, num_classes=100, device='cuda',
                 T=1000, t_start=500, nfe=5, lambda_diff=1.0, lambda_kd=1.0, lambda_kd_logits=1.0):
        super().__init__()
        self.teacher = teacher.eval()                # frozen
        for p in self.teacher.parameters(): p.requires_grad = False

        self.student = student

        self.device = device
        self.sch = DiffSchedule(T=T, device=device)
        self.t_start = t_start
        self.nfe = nfe

        # Feature channels (before avgpool) for your ResNet: last stage is 64 on CIFAR
        self.feat_c = 64

        # Denoisers
        self.phi_feat = DiffusionDenoiser2D(self.feat_c).to(device)
        self.phi_logit = DiffusionDenoiser1D(num_classes).to(device)

        # Noise adapters (γ)
        self.adapt_feat = NoiseAdapter2D(self.feat_c).to(device)
        self.adapt_logit = NoiseAdapter1D(num_classes).to(device)

        # in case teacher/student channels mismatch, add a 1×1 proj for student feature
        self.stu_proj = nn.Conv2d(self.feat_c, self.feat_c, 1).to(device)

        self.lambda_diff = lambda_diff
        self.lambda_kd = lambda_kd
        self.lambda_kd_logits = lambda_kd_logits

    def get_feats_logits(self, model, x):
        feats, logits = model(x, is_feat=True)   # feats: [f0,f1,f2,f3,f4]; we need f3
        f3 = feats[3]                            # [B, C=64, H=8, W=8] on CIFAR
        return f3, logits

    def train_step(self, batch, optimizer):
        x, y = batch
        x = x.to(self.device); y = y.to(self.device)

        # ----- teacher inference (no grad) -----
        with torch.no_grad():
            t_feat, t_logit = self.get_feats_logits(self.teacher, x)
            # losses use these as "clean" targets
            t_feat_detached = t_feat.detach()
            t_logit_detached = t_logit.detach()

        # ----- student forward -----
        s_feat, s_logit = self.get_feats_logits(self.student, x)
        s_feat = self.stu_proj(s_feat)

        # ----- (A) Train denoisers: noise-pred on teacher outputs -----
        B = x.size(0)
        # sample random diffusion times for noise-pred training
        t_rand = torch.randint(1, self.sch.T, (B,), device=self.device)
        abar_t = self.sch.abar_at(t_rand).view(B,1,1,1)
        eps_feat = torch.randn_like(t_feat_detached)
        zt_feat = torch.sqrt(abar_t) * t_feat_detached + torch.sqrt(1.0 - abar_t) * eps_feat
        eps_hat_feat = self.phi_feat(zt_feat, t_rand)
        loss_diff_feat = F.mse_loss(eps_hat_feat, eps_feat)

        # logits branch
        abar_tl = self.sch.abar_at(t_rand).view(B,1)
        eps_logit = torch.randn_like(t_logit_detached)
        zt_logit = torch.sqrt(abar_tl) * t_logit_detached + torch.sqrt(1.0 - abar_tl) * eps_logit
        eps_hat_logit = self.phi_logit(zt_logit, t_rand)
        loss_diff_logit = F.mse_loss(eps_hat_logit, eps_logit)

        loss_diff = loss_diff_feat + loss_diff_logit

        # ----- (B) Denoise student -> KD against teacher -----
        # start from t_start and do nfe jumps down to 0
        with torch.no_grad():
            # precompute the time grid
            steps = torch.linspace(self.t_start, 0, self.nfe+1, device=self.device).long()
        # Feature branch
        gamma_feat = self.adapt_feat(s_feat)                  # [B,1,1,1]
        z = gamma_feat * s_feat + (1.0 - gamma_feat) * torch.randn_like(s_feat)
        for i in range(self.nfe):
            t_cur = steps[i].expand(B)
            t_prev = steps[i+1].expand(B)
            abar_cur = self.sch.abar_at(t_cur).view(B,1,1,1)
            abar_prev = self.sch.abar_at(t_prev).view(B,1,1,1).clamp(min=1e-6)
            eps = self.phi_feat(z, t_cur)
            z = ddim_update(z, eps, abar_cur, abar_prev)
        s_feat_denoised = z
        loss_kd_feat = F.mse_loss(s_feat_denoised, t_feat_detached)

        # Logits branch
        gamma_log = self.adapt_logit(s_logit)                 # [B,1]
        z = gamma_log * s_logit + (1.0 - gamma_log) * torch.randn_like(s_logit)
        for i in range(self.nfe):
            t_cur = steps[i].expand(B)
            t_prev = steps[i+1].expand(B)
            abar_cur = self.sch.abar_at(t_cur).view(B,1)
            abar_prev = self.sch.abar_at(t_prev).view(B,1).clamp(min=1e-6)
            eps = self.phi_logit(z, t_cur)
            # 1D version of update
            x0 = (z - torch.sqrt(1.0 - abar_cur) * eps) / torch.sqrt(abar_cur)
            z  = torch.sqrt(abar_prev) * x0 + torch.sqrt(1.0 - abar_prev) * eps
        s_logit_denoised = z
        # logits KD with KL (temperature=1 per paper baseline on logits)
        loss_kd_log = F.kl_div(
            F.log_softmax(s_logit_denoised, dim=1),
            F.softmax(t_logit_detached, dim=1),
            reduction="batchmean"
        )

        # ----- (C) Task loss (CE) -----
        loss_task = F.cross_entropy(s_logit, y)

        # ----- total -----
        loss = loss_task + self.lambda_diff * loss_diff + self.lambda_kd * loss_kd_feat + self.lambda_kd_logits * loss_kd_log

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            pred = s_logit.argmax(1)
            acc = (pred == y).float().mean().item() * 100.0

        return {
            "loss_total": float(loss),
            "loss_task": float(loss_task),
            "loss_diff": float(loss_diff),
            "loss_kd_feat": float(loss_kd_feat),
            "loss_kd_log": float(loss_kd_log),
            "acc": acc,
        }

# -------------------------
# Training loop for DiffKD
# -------------------------
def train_via_DiffKD(teacher, student, train_loader, test_loader, device,
                     epochs=240, base_lr=0.05, wd=5e-4,
                     t_start=500, nfe=5, save_path="student_diffkd.pth"):
    torch.cuda.empty_cache()
    trainer = DiffKDTrainer(
        teacher.to(device),
        student.to(device),
        num_classes=100,
        device=device,
        T=1000,
        t_start=t_start,
        nfe=nfe,
        lambda_diff=1.0,         # λ1 (diffusion noise-pred losses)
        lambda_kd=1.0,           # λ3 on feature KD (MSE)
        lambda_kd_logits=1.0     # logits KD (KL)
    )

    # optimize student + denoisers + adapters + proj (teacher frozen)
    params = list(trainer.student.parameters()) + \
             list(trainer.phi_feat.parameters()) + list(trainer.phi_logit.parameters()) + \
             list(trainer.adapt_feat.parameters()) + list(trainer.adapt_logit.parameters()) + \
             list(trainer.stu_proj.parameters())

    optimizer = torch.optim.SGD(params, lr=base_lr, momentum=0.9, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150,180,210], gamma=0.1)

    @torch.no_grad()
    def evaluate(model, loader):
        model.eval()
        correct = total = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
        return 100.0 * correct / total

    best = -1.0
    for e in range(1, epochs+1):
        trainer.train()
        running = {"loss_total":0,"acc":0}
        n = 0
        for batch in tqdm(train_loader, desc=f"[DiffKD] Epoch {e}/{epochs}"):
            stats = trainer.train_step(batch, optimizer)
            bs = batch[0].size(0)
            n += bs
            running["loss_total"] += stats["loss_total"] * bs
            running["acc"] += stats["acc"] * bs

        scheduler.step()
        tr_loss = running["loss_total"]/n
        tr_acc  = running["acc"]/n
        te_acc  = evaluate(trainer.student, test_loader)

        print(f"Epoch {e:3d}/{epochs} | loss {tr_loss:.3f} | train {tr_acc:5.2f}% | test {te_acc:5.2f}%")

        if te_acc > best:
            best = te_acc
            state = trainer.student.module.state_dict() if isinstance(trainer.student, nn.DataParallel) else trainer.student.state_dict()
            torch.save(state, save_path)
            print(f"  ↳ Saved best @ epoch {e} (test {best:.2f}%) → {save_path}")

    print(f"✅ DiffKD finished. Best Test Acc: {best:.2f}%")
    return best


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1) Teacher
teacher = resnet56(num_classes=100)
t_ckpt  = torch.load("/kaggle/input/resnet-56/ckpt_epoch_240.pth", map_location=device, weights_only=False)
teacher.load_state_dict(t_ckpt['model'] if 'model' in t_ckpt else t_ckpt)

# 2) Student
student = resnet20(num_classes=100)

# 3) Train with DiffKD (feature+logits, NFEs=5, t_start=500)
train_via_DiffKD(
    teacher, student,
    train_loader, test_loader, device,
    epochs=240, base_lr=0.05, wd=5e-4,
    t_start=500, nfe=5,
    save_path="rn20_from_rn56_DiffKD.pth"
)


[DiffKD] Epoch 1/240:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch   1/240 | loss 18.687 | train  7.53% | test 12.48%
  ↳ Saved best @ epoch 1 (test 12.48%) → rn20_from_rn56_DiffKD.pth


[DiffKD] Epoch 2/240:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch   2/240 | loss 16.571 | train 16.80% | test 19.78%
  ↳ Saved best @ epoch 2 (test 19.78%) → rn20_from_rn56_DiffKD.pth


[DiffKD] Epoch 3/240:   0%|          | 0/391 [00:00<?, ?it/s]

...

[DiffKD] Epoch 239/240:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 239/240 | loss 12.765 | train 85.81% | test 70.10%


[DiffKD] Epoch 240/240:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 240/240 | loss 12.756 | train 85.84% | test 70.17%
✅ DiffKD finished. Best Test Acc: 70.50%


70.5